# 00000: Georeference
Authors: Tobias G. Mueller, Mark A. Buckner
Last modified: 4 Dec 2024
Contact: __________

**Summary**: Here, we georeference the model predictions and orthomosaic


This script outputs 
- a georeference raster in tif format
- a list of gps coordinates for each predicted nest in csv format

The data used in this script was generated in:
    `AIggregation/notebooks/03_validation_and_optimization.ipynb`



In [ ]:
# Imports
import os
import rasterio
from rasterio.plot import show
import numpy as np
import csv
import pandas as pd
import glob

# first check the wd is not notebooks but the main folder
print("cwd is", os.getcwd())

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    print("cwd changed to", os.getcwd())





In [ ]:
# set import and export paths

orthomosaic_path = "datasets/drone_ortho/ortho_clip_23april.png"

prediction_path = "datasets/export_predictions/final"
# path to folder with prediction


In [ ]:


# open ungeoreferenced raster

unRefRaster = rasterio.open(orthomosaic_path)
unRefRaster


# view raster
show(unRefRaster)


# get raster shape to adjust detection points to pixel coordinates
print("image pixel dimensions are:")
print(unRefRaster.read(1).shape)

# save image dimensions
imgwidth = unRefRaster.read(1).shape[1]
imgheight = unRefRaster.read(1).shape[0]


# now insert ground control points matching pixels to gps coordinates
# the best way is to open the uncropped orthomosaic in qgis. This is georeferenced based on the drones GPS
# find GCP coord and matching pixel coordinates

# this georeferences back to the orthophoto and the drone's GPS

# NE corner of Holford plaque
gcp1 = rasterio.control.GroundControlPoint(col = 10304, row=1761, x=379256.53700, y=4699478.52032)

# NE corner of Brooks plaque
gcp2 = rasterio.control.GroundControlPoint(col = 21997, row=7147, x=379262.19952, y=4699478.42842)

# NE corner of Mabee plaque
gcp3 = rasterio.control.GroundControlPoint(col = 21015, row=916, x=379260.699480, y=4699480.76311)

# NE corner of margorie mabee plaque
gcp4 = rasterio.control.GroundControlPoint(col = 28347,row=275, x=379263.53480, y=4699482.32240)

# Stalk of pinecone SW of image
gcp5 = rasterio.control.GroundControlPoint(col = 3379, row=9200, x=379255.0672, y=4699474.2969)




# list of selected gcps
gcp = [gcp1, gcp2, gcp3, gcp4, gcp5]
gcp

# calculate affine transformation from gcps
transformation = rasterio.transform.from_gcps(gcp)
print(" ")
print("affine transformation:")
print(transformation)

#define output raster path
outputPath = "datasets/geo_test/georefRaster.tif"

#create raster and write bands
# by using the transformation and crs 
with rasterio.open(
    outputPath, # specify output raster file path
    'w', # 'w' to specify writing mode
    driver='GTiff', #name of deisred format driver
    height=unRefRaster.read(1).shape[0], #height, in this case taken from unreferenced raster
    width=unRefRaster.read(1).shape[1], #width, in this case taken from unreferenced raster
    count=3, # how many dataset bands (3 for standard color)
    dtype=unRefRaster.read(1).dtype, #data type of dataset
    crs=rasterio.crs.CRS.from_epsg(32618), #CRS.  (WGS84 18N)
    transform=transformation, # specify affine transformation to be used
) as dst: # write out bands
    dst.write(unRefRaster.read(1), 1)
    dst.write(unRefRaster.read(2), 2)
    dst.write(unRefRaster.read(3), 3)

#show georeferenced raster 
geoRaster = rasterio.open(outputPath)
show(geoRaster)



# now use transformation to adjust detections and georef them


# read in detection bounding boxes
# ADJUST THIS TO READ IN WHATEVER THRESHOLDED DETECTION SET WE DECIDE ONE

labels_txt = "datasets/export_predictions/large_slices_large_overlap/ortho_clip_23april.txt"




labels = pd.read_csv(labels_txt, sep=' ',  names=['class', 'x', 'y', 'w', 'h', 'conf'])

#rescale detections from 0-1 (yolo formate) to pixel coordinates
labels[['x', 'w']] = labels[['x', 'w']] * imgwidth
labels[['y', 'h']] = labels[['y', 'h']] * imgheight

# i think yolo x and y are centroids so no need to worry about w and h as belo# now edit and read in a raster of detectionsw
#labels.x = labels.x+(labels.w*0.5)
#labels.y = labels.x-(labels.h*0.5)

centers = labels[['x','y']]
centers['z'] = 1 #add a z value since affine is a 3x3 matrix

# change from type affine to array
tmatrix=  np.array(transformation)
# and specify matrix shape
tmatrix.shape = (3,3)


export_georef_centroids = "datasets/geo_test/georef_centroids.csv"

# multiply csv rows by affine matrix and save
with open(export_georef_centroids, 'w', newline='') as output:
  writer = csv.writer(output)
  writer.writerow(['x','y','z'])
  for index, row in centers.iterrows():
    mrow = np.array(row)
    print(mrow)
    newxy = np.dot(tmatrix,mrow)
    writer.writerow(newxy)